In [63]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
from neo4j import GraphDatabase

In [64]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-


In [65]:
openai = OpenAI()
MODEL = 'gpt-4o-mini'

In [66]:
class Neo4jClient:
    def __init__(self, uri, user, password, db="database"):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        self.db = db

    def close(self):
        self.driver.close()

    def run_query(self, cypher_query):
        with self.driver.session(database=self.db) as session:
            result = session.run(cypher_query)
            return [record.data() for record in result]

In [67]:
query_generator_prompt = """
You are an AI assistant that only converts Turkish user requests into valid Cypher queries for querying a Neo4j graph database.

If the user request is conversational (e.g., greetings, confirmations, acknowledgements like “tamam”, “teşekkürler”, “selam”, etc.), do NOT return a Cypher query.
Instead, return this exact response: []

If the user request is a query about people, connections, or database lookup — only then generate a valid Cypher query (e.g., MATCH...).

DO NOT include explanations for the Cypher queries. Just the query.

The database contains nodes of type `Person` with the following properties:
{
    "ad": "Ahmet Doğan",
    "dogum_yeri": "Kayseri",
    "yas": 37,
    "ilkokul": "Hacı Bektaş-ı Veli İlkokulu",
    "ilkokul_yillari": "1994-2002",
    "lise": "Ankara Fen Lisesi",
    "lise_yillari": "2002-2006",
    "universite": "Orta Doğu Teknik Üniversitesi (ODTÜ)",
    "universite_yillari": "2006-2010",
    "bolum": "Bilgisayar Mühendisliği",
    "yuksek_lisans": "Koç Üniversitesi",
    "yuksek_lisans_yillari": "2010-2012",
    "doktora": null,
    "doktora_yillari": null,
    "calistigi_kurumlar": ["ASELSAN", "SAP Türkiye", "Artemis Yazılım"],
    "calisma_yillari": ["2010-2014", "2014-2017", "2017-devam ediyor"],
    "yasadigi_sehir": "İstanbul",
    "hobiler": ["Tarihi roman okumak", "amatör drone fotoğrafçılığı", "bağlama çalmak", "açık kaynak yazılım projelerine katkıda bulunmak"],
    "es": "Elif Doğan",
    "cocuklar": ["Deniz Doğan", "Ada Doğan"],
    "dosya_adi": "ahmet_dogan.md"
}

Relationships between people use the `RELATED` relationship type.
The meaning of each connection is stored in a `type` property (e.g., 'UNIVERSITE_ARKADASI', 'AYNI_MEMLEKETTEN').

Relationships between people use the `RELATED` relationship type with a `type` property (e.g., 'UNIVERSITE_ARKADASI', 'AYNI_MEMLEKETTEN').

Example:
User: Ahmet Doğan ile aynı üniversitede okuyan kişileri göster.
Output: MATCH (p1:Person {ad: 'Ahmet Doğan'})<-[:RELATED]-(p2:Person) WHERE p1.universite = p2.universite RETURN DISTINCT p2

User: selam nasılsın?
Output: []
"""





result_interpreter_prompt = ""
result_interpreter_prompt += "You are a helpful assistant that receives structured data (such as Neo4j query results) and explains the findings in natural, fluent Turkish."

result_interpreter_prompt += " Your input will be a Neo4j response in the form of paths, nodes, or JSON structure that represent how people are connected."

result_interpreter_prompt += " Based on the graph data, summarize the path in Turkish and explain how the source person can reach the target person. Mention mutual schools, work places, or hometown connections if relevant."

result_interpreter_prompt += " Be concise, natural, and helpful. Do not invent or hallucinate any information. Base your answer strictly on the graph data."

result_interpreter_prompt += " Example input: A graph showing Gülnur Yıldız → Gül Yıldırım (üniversite arkadaşı) → Ali Türkmen (hemşehrisi) → Ahmet Doğan (lise arkadaşı, ASELSAN çalışanı)."
result_interpreter_prompt += " Output: Elimdeki bilgilere göre Gül Yıldırım'ın üniversite arkadaşı olduğunu görüyorum. Gül Yıldırım'ın hemşehrisi Ali Türkmen var. Ali Türkmen'in lise arkadaşı ise Ahmet Doğan ve şu anda ASELSAN'da çalışıyor."

result_interpreter_prompt += " Important: Your output must always be in Turkish."



In [68]:
def chat_1(message):
    messages = [
        {"role": "system", "content": query_generator_prompt},
        {"role": "user", "content": message}
    ]
    completion = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
    )
    query = completion.choices[0].message.content.strip()
    return query

In [69]:
print(chat_1("Tüm alperen leri getir"))

MATCH (p:Person) WHERE p.ad CONTAINS 'Alperen' RETURN p


In [70]:
cypher_query = chat_1("Tüm alperen leri getir")
neo4j_client = Neo4jClient(
    uri="bolt://localhost:7687", 
    user="neo4j", 
    password="alperen2239",
    db="database"
)

if cypher_query.lower().__contains__("cypher query:"):
        cypher_query = cypher_query.split(":", 1)[1].strip()

result = neo4j_client.run_query(cypher_query)
print(result)

[{'p': {'yuksek_lisans_yillari': '2009-2011', 'ad': 'Alperen Aydın', 'hobiler': ['amatör fotoğrafçılık', 'doğa fotoğrafçılığı', 'sokak fotoğrafçılığı', 'uzun yürüyüşler', 'kampçılık', 'eski Türk filmlerini izlemek', 'mutfakta yeni yemek tarifleri denemek'], 'yasadigi_sehir': 'İstanbul', 'yas': 40, 'ilkokul': 'Mustafa Kemal İlkokulu', 'cocuklar': ['Deniz', 'Can'], 'yuksek_lisans': 'Bahçeşehir Üniversitesi', 'universite_yillari': '2003-2007', 'bolum': 'Halkla İlişkiler ve Tanıtım', 'dogum_yeri': 'İzmir', 'universite': 'Ege Üniversitesi', 'calistigi_kurumlar': ['Marka İletişim Ajansı', 'Trendyol', 'Turkcell', 'Hepsiburada', 'Dijital Köprü'], 'ilkokul_yillari': '1991-1996', 'lise_yillari': '1996-2003', 'name': 'Alperen Aydın', 'calisma_yillari': ['2007-2009', '2009-2013', '2013-2017', '2017-2021', '2021-devam ediyor'], 'lise': 'İzmir Bornova Anadolu Lisesi', 'dosya_adi': 'alperen_aydin.md'}}, {'p': {'yuksek_lisans_yillari': '2013-2015', 'ad': 'Alperen Baran', 'hobiler': ['koşu yapmak', 'do

In [71]:
def chat_2(history, user_message, neo4j_result):
    if neo4j_result == []:
        # Grafik verisi yoksa, sadece kullanıcı mesajına tepki ver
        messages = [{"role": "system", "content": "Sen kullanıcıyla doğal, akıcı Türkçe'de sohbet eden bir asistansın. Kullanıcı Neo4j sorgusu değil, muhtemelen selamlaşma, sohbet veya başka bir gündelik şey yazmıştır. Bu mesajlara uygun yanıt ver."}]
        messages += history + [{"role": "user", "content": user_message}]
    else:
        # Grafik verisi varsa, bunu yorumlayan sistem prompt kullanılır
        messages = [{"role": "system", "content": result_interpreter_prompt}]
        messages += history + [{"role": "user", "content": f"Original question: {user_message}\nQuery Result: {neo4j_result}"}]

    completion = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
    )
    return completion.choices[0].message.content


In [72]:
def chat_pipeline(message, history):
    cypher_query = chat_1(message)

    if cypher_query.strip() == "[]":
        return chat_2(history, message, [])

    neo4j_client = Neo4jClient(
        uri="bolt://localhost:7687",
        user="neo4j",
        password="alperen2239",
        db="database"
    )

    result = neo4j_client.run_query(cypher_query)
    neo4j_client.close()

    response = chat_2(history, message, result)
    return response


In [73]:
gr.ChatInterface(fn=chat_pipeline, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.
